**<font size="5"><b>Project Name - PlayStore_App_Review_Analysis</b></font>**

Project Type - EDA

Contribution - Individual


**<font size="5"><b>Problem Statement</b></font>**

The Play Store apps data has enormous potential to drive app-making businesses to success. Actionable insights can be drawn for developers to work on and capture the Android market. Each app (row) has values for category, rating, size, and more. Another dataset contains customer reviews of the android apps. Explore and analyse the data to discover key factors responsible for app engagement and success.

**Hypothesis / Assumptions**


*   Free apps have more installs, while paid apps may get higher ratings.
*   More reviews and positive sentiments lead to better ratings.
*   Larger app sizes might reduce installs.
*   Games, Communication, and Tools dominate in installs, recently updated apps tend to have higher ratings.



**<font size="6"><b>Let's Begin!</b></font>**

**Import Libraries**

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display all columns for better viewing
pd.set_option('display.max_columns', None)

# For better plot visuals
sns.set(style='whitegrid', palette='muted', font_scale=1.1)

# To display plots inline
%matplotlib inline
#pandas → For data manipulation (loading, cleaning, analyzing)
#numpy → For numerical operations (conversion, calculations)
#matplotlib.pyplot → For visualizations and graphs
#seaborn → For advanced, beautiful plots (works on top of matplotlib)


**Load the Datasets**

In [8]:
apps_df = pd.read_csv('/content/Play Store Data.csv')

# Load User Reviews Data
reviews_df = pd.read_csv('/content/User Reviews.csv')

# Display success message
print("Play Store Data Loaded Successfully with shape:", apps_df.shape)
print("User Reviews Data Loaded Successfully with shape:", reviews_df.shape)
#we need to add the required data set files in the colab format so that it will read it directly without errors

Play Store Data Loaded Successfully with shape: (10841, 13)
User Reviews Data Loaded Successfully with shape: (64295, 5)


**Dataset View**


In [ ]:
print("Play Store Data - first 5 rows")
display(apps_df.head())

print("\nUser Reviews - first 5 rows")
display(reviews_df.head())

#shape (rows, cols)
print("\nPlay Store Data shape:", apps_df.shape)
print("User Reviews shape:", reviews_df.shape)

#Info (types & non-null counts)
print("\nPlay Store Data info():")
apps_df.info()

print("\nUser Reviews info():")
reviews_df.info()

#Summary statistics
print("\nPlay Store Data - numeric describe():")
display(apps_df.describe())

print("\nPlay Store Data - all columns describe():")
display(apps_df.describe(include='all').T)   # transpose for readability

print("\nUser Reviews - describe():")
display(reviews_df.describe(include='all').T)

#Duplicate rows count
dup_apps = apps_df.duplicated().sum()
dup_reviews = reviews_df.duplicated().sum()
print(f"\nDuplicate rows -> Play Store Data: {dup_apps} | User Reviews: {dup_reviews}")

#Top values for some key categorical cols (quick check)
for col in ['Category', 'Type', 'Content Rating', 'Genres']:
    if col in apps_df.columns:
        print(f"\nTop values for {col}:")
        display(apps_df[col].value_counts().head(10))

#Quick missing-value counts by column (sorted)
print("\nMissing values in Play Store Data (per column):")
display(apps_df.isnull().sum().sort_values(ascending=False))

print("\nMissing values in User Reviews (per column):")
display(reviews_df.isnull().sum().sort_values(ascending=False))

**<font size="5"><b>Duplicate Handling and Missing Values</b></font>**

In [ ]:
#Duplicate handling
# Count duplicates
dup_apps = apps_df.duplicated().sum()
dup_reviews = reviews_df.duplicated().sum()
print("Duplicate rows before drop -> Apps:", dup_apps, "| Reviews:", dup_reviews)

#need few duplicate examples
if dup_apps > 0:
    display(apps_df[apps_df.duplicated()].head())

if dup_reviews > 0:
    display(reviews_df[reviews_df.duplicated()].head())

# Drop duplicates (recommended) and confirm
apps_df = apps_df.drop_duplicates().reset_index(drop=True)
reviews_df = reviews_df.drop_duplicates().reset_index(drop=True)
print("Dropped duplicates. New shapes -> Apps:", apps_df.shape, "| Reviews:", reviews_df.shape)

# Missing counts per column
print("\nMissing values per column - Apps:")
missing_apps = apps_df.isna().sum().sort_values(ascending=False)
display(missing_apps)

print("\nMissing values per column - Reviews:")
missing_reviews = reviews_df.isna().sum().sort_values(ascending=False)
display(missing_reviews)

# Missing value percentages
print("\nMissing % - Apps:")
display((apps_df.isna().mean()*100).sort_values(ascending=False).round(2))

print("\nMissing % - Reviews:")
display((reviews_df.isna().mean()*100).sort_values(ascending=False).round(2))


**Data Visualization**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
sns.heatmap(apps_df.isnull(), cbar=False)
plt.title("Missing values heatmap - Apps")
plt.show()

plt.figure(figsize=(8,4))
missing_apps[missing_apps>0].plot(kind='bar')
plt.title("Missing value counts by column - Apps")
plt.ylabel("Number of missing")
plt.show()

plt.figure(figsize=(12,6))
sns.heatmap(reviews_df.isnull(), cbar=False)
plt.title("Missing values heatmap - Reviews")
plt.show()

plt.figure(figsize=(8,4))
missing_reviews[missing_reviews>0].plot(kind='bar')
plt.title("Missing value counts by column - Reviews")
plt.ylabel("Number of missing")
plt.show()


**Data Wrangling**

In [ ]:
apps = apps_df.copy()
reviews = reviews_df.copy()

# 1. Convert Rating & Reviews to numeric
apps['Rating'] = pd.to_numeric(apps['Rating'], errors='coerce')
apps['Reviews'] = pd.to_numeric(apps['Reviews'].str.replace(',', ''), errors='coerce')

# 2. Parse Installs -> numeric
def parse_installs(x):
    try:
        if pd.isna(x): return np.nan
        s = str(x).strip().replace('+','').replace(',','')
        return int(s)
    except:
        return np.nan

apps['Installs_num'] = apps['Installs'].apply(parse_installs)

# 3. Parse Size -> MB
def parse_size_to_mb(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s.lower() == 'varies with device':
        return np.nan
    try:
        if s.endswith('M'):
            return float(s[:-1])
        if s.endswith('k') or s.endswith('K'):
            return float(s[:-1]) / 1024
        s2 = s.replace(',','')
        val = float(s2)
        if val > 1000:
            return val / (1024*1024)
        return val / (1024*1024)
    except:
        return np.nan

apps['Size_mb'] = apps['Size'].apply(parse_size_to_mb).round(3)

# 4. Price -> numeric
def parse_price(p):
    if pd.isna(p): return 0.0
    s = str(p).strip().replace('$','').replace(',','')
    if s == '' or s.lower() == 'free':
        return 0.0
    try:
        return float(s)
    except:
        return 0.0

apps['Price_num'] = apps['Price'].apply(parse_price)

# 5. Derived columns
apps['is_free'] = apps['Price_num'] == 0
apps['log_installs'] = np.log1p(apps['Installs_num'].fillna(0))

# 6. Convert 'Last Updated' to datetime
apps['Last Updated'] = pd.to_datetime(apps['Last Updated'], errors='coerce')
apps['last_update_year'] = apps['Last Updated'].dt.year
apps['last_update_month'] = apps['Last Updated'].dt.month

# 7. Clean Genres
apps['primary_genre'] = apps['Genres'].astype(str).apply(lambda x: x.split(';')[0].split(',')[0])

# 8. Merge sentiment polarity from reviews
if 'Sentiment_Polarity' in reviews.columns:
    reviews['Sentiment_Polarity'] = pd.to_numeric(reviews['Sentiment_Polarity'], errors='coerce')
    sentiment_agg = reviews.groupby('App')['Sentiment_Polarity'].agg(['mean','count']).reset_index()
    sentiment_agg.rename(columns={'mean':'avg_sentiment_polarity','count':'review_count'}, inplace=True)
    apps = apps.merge(sentiment_agg, on='App', how='left')

# 9. Quick check of new columns
print("After cleaning shapes -> apps:", apps.shape)
display(apps[['App','Rating','Reviews','Installs_num','Size_mb','Price_num','log_installs','avg_sentiment_polarity']].head())

# 10. Save cleaned dataset for visualization step
apps.to_csv('playstore_apps_cleaned.csv', index=False)
print("Saved cleaned file as playstore_apps_cleaned.csv")

**<font size="5"><b>EDA and Visualization</b></font>**

**Distribution of App Ratings**

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(apps['Rating'], bins=20, kde=True)
plt.title('Distribution of App Ratings')
plt.xlabel('Rating')
plt.ylabel('Number of Apps')
plt.show()
##Most apps have ratings between 4.0 and 4.5, users generally rate apps positively and also very few apps have ratings below 3.0.

**Categories by Number of Apps**

In [ ]:
top_categories = apps['Category'].value_counts().head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_categories.values, y=top_categories.index)
plt.title('Top 10 App Categories by Count')
plt.xlabel('Number of Apps')
plt.ylabel('Category')
plt.show()
#The largest number of apps fall under “Family”, “Game”, and “Tools” categories. These dominate the Play Store ecosystem.

**free and paid -> Average Rating Comparison**

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='is_free', y='Rating', data=apps)
plt.title('Free vs Paid Apps: Rating Comparison')
plt.xlabel('Is the App Free?')
plt.ylabel('Rating')
plt.xticks([0,1], ['Paid', 'Free'])
plt.show()
##Paid apps tend to have slightly higher average ratings than free apps, possibly because paying users expect better quality and fewer ads.

**Top 10 Categories by Installs**

In [ ]:
top_installs = apps.groupby('Category')['Installs_num'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_installs.values, y=top_installs.index)
plt.title('Top 10 Categories by Total Installs')
plt.xlabel('Total Installs')
plt.ylabel('Category')
plt.show()


**Correlation Heatmap**

In [ ]:
plt.figure(figsize=(10,7))
numeric_cols = ['Rating','Reviews','Installs_num','Size_mb','Price_num','log_installs','avg_sentiment_polarity']
corr = apps[numeric_cols].corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Heatmap (Numeric Features)')
plt.show()
#Ratings have moderate correlation with Reviews and Installs, suggesting that popular apps often maintain good ratings.

**<font size="5"><b>Business Insights and Recommendations</b></font>**


*   Categories such as Games, Communication, and Social have the highest total installs. These categories dominate user engagement on the Play Store.
*   Free apps make up the majority of the Play Store and get more installs, while paid apps generally maintain slightly higher ratings because they offer better quality and have fewer ads.


*  Apps with a higher number of reviews and installs usually maintain better ratings. This shows that popularity and trust often go hand in hand with perceived quality.
*  App size and price have very little effect on ratings, although very large apps (more than 100 MB) tend to have fewer installs.

* Apps that have more positive review sentiments also have higher ratings, which shows that user satisfaction directly impacts success.

**Recommendations for stakeholders**



*   Developers in high-install categories like Games and Communication should focus on regular updates, bug fixes, and feature improvements to retain users.

*   Paid app developers should ensure a premium experience, such as fewer ads, smoother performance, and new features that justify the price.

*   Encourage users to leave reviews and ratings after positive experiences to increase visibility and social proof.
*   Try to reduce the app’s size to make it more accessible to users with limited storage or data.


*   Monitor user reviews frequently, study the sentiment patterns, and quickly fix issues that users mention repeatedly.


<font size="5"><b>Conclusion</b></font>




*   This Exploratory Data Analysis on the Google Play Store dataset provided valuable insights into the key factors influencing app performance and user satisfaction

*   The findings revealed that Games, Communication, and Social categories dominate in both installs and user engagement, indicating these segments drive the Play Store ecosystem.
*   While free apps attract a much larger user base, paid apps generally maintain higher ratings, reflecting better user experience and premium quality.

*   A clear positive relationship was observed between user review sentiment and app ratings, proving that user satisfaction is a crucial driver of success.
These insights can guide developers, marketers, and product managers to make data-driven decisions—optimizing app design, pricing strategies, and user engagement efforts to achieve higher retention and ratings.

Hurrah! You have successfully completed your EDA Capstone Project. 🎯



**GitHub Link**

https://github.com/DanishMullaData/PlayStore_App_Review_Analysis